In [62]:
from typing import Tuple, List

In [63]:
stats_ab = {"visited": 0, "cutoffs": 0}
stats_mm = {"visited": 0}

In [64]:
def terminal_state(board):
    for heap in board:
        if heap != 0:
            return False
    return True

In [65]:
def evaluate(position, is_max_turn):
    if is_max_turn:
        return -1
    return 1

In [66]:
def expand(state):
    result = []
    for idx in range(len(state)):
        heap_size = state[idx]
        if heap_size > 0:
            for take in range(1, heap_size + 1):
                updated = list(state)
                updated[idx] = heap_size - take
                result.append(tuple(updated))
    return result

In [67]:
def run_minimax(node, is_max_turn):
    stats_mm["visited"] += 1

    if terminal_state(node):
        return evaluate(node, is_max_turn)

    if is_max_turn:
        best_score = float("-inf")
        for nxt in expand(node):
            score = run_minimax(nxt, False)
            if score > best_score:
                best_score = score
        return best_score
    else:
        best_score = float("inf")
        for nxt in expand(node):
            score = run_minimax(nxt, True)
            best_score = min(best_score, score)
        return best_score


In [68]:
def alpha_beta_search(node, is_maximizer, a, b):
    stats_ab["visited"] += 1

    if terminal_state(node):
        return evaluate(node, is_maximizer)

    if is_maximizer:
        best_val = float("-inf")
        for child in expand(node):
            score = alpha_beta_search(child, False, a, b)
            best_val = max(best_val, score)
            a = max(a, best_val)
            if a >= b:  # prune
                stats_ab["cutoffs"] += 1
                break
        return best_val
    else:
        best_val = float("inf")
        for child in expand(node):
            score = alpha_beta_search(child, True, a, b)
            if score < best_val:
                best_val = score
            b = min(b, best_val)
            if a >= b:  # prune
                stats_ab["cutoffs"] += 1
                break
        return best_val


In [69]:
def find_best_move_alphabeta(initial_state):
    best_val = -float("inf")
    best_move = None
    alpha = -float("inf")
    beta = float("inf")
    for succ in expand(initial_state):
        val = alpha_beta_search(succ, False, alpha, beta)
        if val > best_val:
            best_val = val
            best_move = succ
        if best_val > alpha:
            alpha = best_val
    return best_move, int(best_val)

In [70]:
def find_best_move_minimax(initial_state):
    best_val = -float("inf")
    best_move = None
    for succ in expand(initial_state):
        val = run_minimax(succ, False)
        if val > best_val:
            best_val = val
            best_move = succ
    return best_move, int(best_val)

In [71]:
if __name__ == "__main__":
    start_position = (3, 4, 5)

    stats_ab["visited"] = 0
    stats_ab["cutoffs"] = 0
    move_ab, score_ab = find_best_move_alphabeta(start_position)

    stats_mm["visited"] = 0
    move_mm, score_mm = find_best_move_minimax(start_position)

    print(f"Starting state: {start_position}\n")

    print("Alpha-Beta Search:")
    print(f"  Move chosen: {start_position} -> {move_ab}")
    print(f"  Value: {score_ab}")
    print(f"  Nodes visited: {stats_ab['visited']}")
    print(f"  Cutoffs: {stats_ab['cutoffs']}\n")

    print("Minimax Search:")
    print(f"  Move chosen: {start_position} -> {move_mm}")
    print(f"  Value: {score_mm}")
    print(f"  Nodes visited: {stats_mm['visited']}\n")

    reduction = stats_mm["visited"] - stats_ab["visited"]
    print("Summary:")
    print(f"  Alpha-Beta visited {stats_ab['visited']} nodes, with {stats_ab['cutoffs']} cutoffs.")
    print(f"  Minimax visited {stats_mm['visited']} nodes.")
    print(f"  Search reduction: {reduction} nodes saved by pruning.")


Starting state: (3, 4, 5)

Alpha-Beta Search:
  Move chosen: (3, 4, 5) -> (1, 4, 5)
  Value: 1
  Nodes visited: 33915
  Cutoffs: 15890

Minimax Search:
  Move chosen: (3, 4, 5) -> (1, 4, 5)
  Value: 1
  Nodes visited: 1038767

Summary:
  Alpha-Beta visited 33915 nodes, with 15890 cutoffs.
  Minimax visited 1038767 nodes.
  Search reduction: 1004852 nodes saved by pruning.
